# 使用 Semantic Kernel 集成 OpenBnB MCP 服务器

本笔记展示了如何使用 Semantic Kernel 与实际的 OpenBnB MCP 服务器集成，通过 MCPStdioPlugin 搜索真实的 Airbnb住宿。对于 LLM 访问，它使用了 Azure AI Foundry。要设置您的环境变量，可以参考 [设置课程](/00-course-setup/README.md)。


## 导入所需的包


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio
import subprocess
import sys


from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## 创建 MCP 插件连接

我们将使用 MCPStdioPlugin 连接到 [OpenBnB MCP 服务器](https://github.com/openbnb-org/mcp-server-airbnb)。该服务器通过 @openbnb/mcp-server-airbnb 包提供 Airbnb 搜索功能。


## 创建客户端

在本示例中，我们将使用 Azure AI Foundry 来访问 LLM。请确保您的环境变量已正确设置。


## 环境配置

配置 Azure OpenAI 设置。确保已设置以下环境变量：
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## 了解 OpenBnB MCP 集成

此笔记本连接到**真实的 OpenBnB MCP 服务器**，提供实际的 Airbnb 搜索功能。

### 工作原理：

1. **MCPStdioPlugin**：通过标准输入/输出与 MCP 服务器进行通信  
2. **真实的 NPM 包**：通过 npx 下载并运行 `@openbnb/mcp-server-airbnb`  
3. **实时数据**：从 Airbnb 的 API 返回真实的房源数据  
4. **功能发现**：代理会自动发现 MCP 服务器提供的可用功能  

### 可用功能：

OpenBnB MCP 服务器通常提供以下功能：  
- **search_listings** - 根据位置和条件搜索 Airbnb 房源  
- **get_listing_details** - 获取特定房源的详细信息  
- **check_availability** - 检查特定日期的房源可用性  
- **get_reviews** - 获取房源的评论  
- **get_host_info** - 获取房源主人的信息  

### 前置条件：

- 系统已安装 **Node.js**  
- **互联网连接**，用于下载 MCP 服务器包  
- **NPX** 可用（随 Node.js 一起安装）  

### 测试连接：

您可以通过运行以下命令手动测试 MCP 服务器：  
```bash
npx -y @openbnb/mcp-server-airbnb
```  

此命令将下载并启动 OpenBnB MCP 服务器，随后 Semantic Kernel 会连接到该服务器以获取真实的 Airbnb 数据。  


## 运行与 OpenBnB MCP 服务器连接的代理

现在我们将运行与 OpenBnB MCP 服务器连接的 AI 代理，用于搜索斯德哥尔摩适合2名成人和1名儿童的真实Airbnb住宿。您可以随意修改 `user_inputs` 列表来更改搜索条件。


In [ ]:
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        print("🚀 Starting with Azure OpenAI...")
        
        # Verify environment variables
        print("🔍 Checking Azure environment variables...")
        required_vars = ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME", "AZURE_OPENAI_ENDPOINT", "AZURE_OPENAI_API_KEY"]
        for var in required_vars:
            if os.getenv(var):
                print(f"✅ {var} is set")
            else:
                print(f"❌ {var} is NOT set")
        
        print("\n🔧 Creating MCP Plugin...")
        
        # Create MCP plugin connection to real OpenBnB server
        # Based on the GitHub repo, the server doesn't need special env vars
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb"],
        ) as airbnb_plugin:

            print("✅ MCP Plugin created and connected")
            
            # Wait a moment for the server to fully initialize
            await asyncio.sleep(2)
            
            # Try to list available tools
            try:
                tools = await airbnb_plugin.get_tools()
                print(f"🔧 Available tools: {[tool.name for tool in tools]}")
            except Exception as e:
                print(f"⚠️ Could not list tools: {str(e)}")

            # Create the Azure OpenAI service with proper configuration
            print("\n🤖 Creating Azure OpenAI service...")
            service = AzureChatCompletion(
                deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
                endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
                api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            )
            
            # Create agent with the service instance
            agent = ChatCompletionAgent(
                service=service,
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the available functions to search for properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("✅ Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 User: {user_input}")
                
                try:
                    # Use the simpler get_response method
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    
                    # Process the response text
                    response_text = str(response)
                    
                    # Remove any markdown code blocks around HTML
                    response_text = response_text.replace('```html', '').replace('```', '')
                    
                    # Display the result
                    print(f"🤖 {response.name}: {response_text[:200]}..." if len(response_text) > 200 else response_text)
                    
                    # If response contains HTML table, display it properly
                    if '<table' in response_text.lower():
                        # Add CSS styling for better table rendering
                        table_css = """
                        <style>
                            .airbnb-results table {
                                border-collapse: collapse;
                                width: 100%;
                                margin: 10px 0;
                            }
                            .airbnb-results th, .airbnb-results td {
                                border: 1px solid #ddd;
                                padding: 8px;
                                text-align: left;
                            }
                            .airbnb-results th {
                                background-color: #f2f2f2;
                                font-weight: bold;
                            }
                            .airbnb-results tr:nth-child(even) {
                                background-color: #f9f9f9;
                            }
                            .airbnb-results a {
                                color: #1976d2;
                                text-decoration: none;
                            }
                            .airbnb-results a:hover {
                                text-decoration: underline;
                            }
                        </style>
                        """
                        html_output = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                        display(HTML(html_output))
                    else:
                        # Display as regular text if no table
                        display(HTML(f'<div class="airbnb-results">{response_text}</div>'))
                        
                except Exception as e:
                    print(f"❌ Error processing user input: {str(e)}")
                    import traceback
                    traceback.print_exc()
                
            # Cleanup
            if thread:
                await thread.delete()
                print("🧹 Thread cleaned up")
                
    except Exception as e:
        print(f"❌ Main error: {str(e)}")
        import traceback
        traceback.print_exc()

# Run the main function
print("🚀 Starting MCP Agent...")
await main()
print("✅ Done!")

# 概要
恭喜你！你已经成功构建了一个能够通过模型上下文协议（MCP）与现实世界住宿搜索集成的 AI 代理：

## 使用的技术：
- Semantic Kernel - 用于使用 Azure OpenAI 构建智能代理
- Azure AI Foundry - 提供大语言模型功能和聊天补全
- MCP（模型上下文协议） - 用于标准化工具集成
- OpenBnB MCP Server - 提供真实的 Airbnb 搜索功能
- Node.js/NPX - 用于运行外部 MCP 服务器

## 你学到了什么：
- MCP 集成：将 Semantic Kernel 代理连接到外部 MCP 服务器
- 实时数据访问：通过实时 API 搜索实际的 Airbnb 房源
- 协议通信：使用标准输入输出（stdio）在代理和 MCP 服务器之间通信
- 功能发现：自动发现 MCP 服务器提供的可用功能
- 流式响应：实时捕获并记录函数调用
- HTML 渲染：以样式化表格和交互式显示格式化代理响应

## 下一步：
- 集成更多 MCP 服务器（如天气、航班、餐厅）
- 构建结合 MCP 和 A2A 协议的多代理系统
- 为你的数据源创建自定义 MCP 服务器
- 实现跨会话的持久对话记忆
- 将代理部署到 Azure Functions，并实现 MCP 服务器编排
- 添加用户认证和预订功能



---

**免责声明**：  
本文档使用AI翻译服务 [Co-op Translator](https://github.com/Azure/co-op-translator) 进行翻译。尽管我们努力确保翻译的准确性，但请注意，自动翻译可能包含错误或不准确之处。应以原始语言的文档作为权威来源。对于重要信息，建议使用专业人工翻译。我们不对因使用此翻译而产生的任何误解或误读承担责任。
